In [5]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import pandas as pd
import pickle
import yaml

import bacco
import baccoemu

import sys
sys.path.append('/dipc/kstoreyf/muchisimocks/scripts')
import data_loader
import plotter
import plot_utils
import utils
import generate_params_lh as gplh

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%matplotlib inline
mpl.pyplot.style.use('default')
mpl.pyplot.close('all')

font, rcnew = plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
mpl.pyplot.rcParams.update(rcnew)
mpl.pyplot.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

In [ ]:
tag_params = '_quijote_p0_n1000'
tag_biasparams = '_b1000_p0_n1'
n_rlzs_per_cosmo = 1

# Parameters to vary
n_cosmo_params_vary = 5  # Number of cosmological parameters to vary
n_bias_params_vary = 0  # Number of bias parameters to vary
cosmo_param_names_vary = utils.cosmo_param_names_ordered[:n_cosmo_params_vary]
bias_param_names_vary = utils.biasparam_names_ordered[:n_bias_params_vary]

In [6]:
config = '../configs/configs_runlike/config_muchisimocks_pk_quijote_p0_n1000_b1000_p0_n1_mean_pvary5_bvary0.yaml'
with open(config, "r") as file:
    config = yaml.safe_load(file)

In [7]:
mcmc_framework = config.get('mcmc_framework', 'dynesty')
idxs_obs = config.get('idxs_obs', [0])
evaluate_mean = config.get('evaluate_mean', False)
data_mode = config['data_mode']
statistics = config['statistics']
tag_params = config['tag_params']
tag_biasparams = config['tag_biasparams']
tag_data = config.get('tag_data', None)
tag_inf = config['tag_inf']
cosmo_param_names_vary = config.get('cosmo_param_names_vary', [])
bias_param_names_vary = config.get('bias_param_names_vary', [])
kwargs_data = config.get('kwargs_data', {})

In [18]:
data_mode = 'muchisimocks'
# tag_params = '_p5_n10000'
# tag_biasparams = '_biaszen_p4_n10000'
n_train = 10000
statistics = ['pk']
tag_stats = "_".join(statistics)

# muchisimocks_pk_p5_n10000_biaszen_p4_n10000_ntrain10000
tags_inf = [
             #f'_{data_mode}{tag_stats}{tag_params}{tag_biasparams}_ntrain{n_train}' for tag_stats in tag_stats_arr
             tag_inf,
            ]
inf_methods = ['dynesty']*len(tags_inf)
labels_extra = [
            '',
                ]

In [19]:
# load fixed-cosmo test set
tag_params_test = '_quijote_p0_n1000'
tag_biasparams_test = '_b1000_p0_n1'
tag_datagen_test = ''
theta_test_fixed = data_loader.load_theta_test(tag_params_test, tag_biasparams_test, 
                                cosmo_param_names_vary=cosmo_param_names_vary, bias_param_names_vary=bias_param_names_vary)
param_names_vary = cosmo_param_names_vary + bias_param_names_vary
print(theta_test_fixed.shape)

# get bounds
_, dict_bounds_cosmo, _ = gplh.define_LH_cosmo(tag_params)
_, dict_bounds_bias, _ = gplh.define_LH_bias(tag_biasparams)
extents = {**dict_bounds_cosmo, **dict_bounds_bias}

(5,)


In [20]:
colors = ['purple']

In [21]:
print(theta_test_fixed)
idx_obs = 0
title = 'recovery test: mean of 1000 fixed cosmo mocks'
tags_test = [tag_inf]


#tags_test = [f'{tag_data_test_fixed}_mean']*len(tags_inf)
# doing utils.biasparam_names_ordered here instead of bias_param_names_vary in case some param sets don't vary all the bias params but we want to show them on the same plot
param_names_show = config['cosmo_param_names_vary']
theta_test_fixed_show = theta_test_fixed[[i for i, name in enumerate(param_names_vary) if name in param_names_show]]
plotter.plot_contours_inf(param_names_show, idx_obs, theta_test_fixed_show, 
                        inf_methods, tags_inf, tags_test=tags_test,
                        colors=colors, labels_extra=labels_extra,
                        title=title, extents=extents)

[0.3175 0.834  0.6711 0.9624 0.049 ]


FileNotFoundError: [Errno 2] No such file or directory: '../results/results_dynesty/samplers_muchisimocks_pk_quijote_p0_n1000_b1000_p0_n1_mean_pvary5_bvary0/sampler_results_idx0.npy'